In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1663909989453_0033,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
Q400Final = spark.read.csv('s3://sjet-datamart-bucket/PoC-Ayush/Test5/', header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
Q400_stage3_df = Q400Final.filter(~(F.col("Radio-Altitude").like ("*")) 
                                                        & (F.col("Radio-Altitude") != " "))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
Q400Final.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

171040

In [6]:
Q400Interim = Q400_stage3_df.alias('ED').select(
                                                        'ED.FlightID',
                                                        'ED.DesigFlightNumber',
                                                        'ED.FlightNumberARMS',
                                                        'ED.FlightDate',
                                                        'ED.FromCity',
                                                        'ED.ToCity',
                                                        'ED.BlockTime',
                                                        'ED.ECN',
                                                        'ED.EmployeeName',
                                                        'ED.RANK',
                                                        'ED.ChocksOff',
                                                        'ED.ChocksOn',
                                                        'ED.DOMINT',
                                                        'ED.AircraftType',
                                                        'ED.AircraftModel',  
                                                        'ED.Payload',
                                                        'ED.TailNo',
                                                        'ED.Departure',
                                                        'ED.Arrival',
                                                        'ED.FlightNo',
                                                        'ED.Date',
                                                         F.regexp_replace(
                                                               F.col('ED.Time-In-Secs'), '\\*', '').cast('Double').alias('Time-In-Secs'),
                                                        'ED.Time-Per-Sec', 
                                                        'ED.BARO-CORRECTED-PALT-DERIVED', 
                                                        'ED.Altitude-BaroL', 
                                                        'ED.Magnetic-Heading', 
                                                        F.lit(F.regexp_replace(
                                                        'ED.Pitch-Attitude', '\\*', '')).alias('Pitch-Attitude'),
                                                        'ED.Roll-Attitude',  
                                                        F.col('ED.Radio-Altitude').cast('Integer').alias('Radio-Altitude'),
                                                        F.lit(F.regexp_replace('ED.Nose-WOW', '\\*', '')).alias('Nose-WOW'), 
                                                        F.lit(F.regexp_replace('ED.Main-WOW', '\\*', '')).alias('Main-WOW'),
                                                        'ED.Normal-Acceleration', 
                                                        'ED.Lateral-Acceleration', 
                                                        'ED.Longitudinal-Acceleration', 
                                                        'ED.Calibrated-Airspeed-L', 
                                                        'ED.AFCS-Selected-Speed', 
                                                        'ED.Vertical-Speed-Derived', 
                                                        'ED.Ground-Speed', 
                                                        'ED.Flap-Control-Selection',
                                                        F.regexp_replace(F.col('ED.FLAP-POSITION'), '\\*', '').alias('FLAP-POSITION'), 
                                                        'ED.Power-Lever-Pos-L', 
                                                        'ED.Power-Lever-Pos-R', 
                                                        'ED.Torque-TQ-Eng-L', 
                                                        'ED.Torque-TQ-Eng-R', 
                                                        'ED.Condition-Lever-L', 
                                                        'ED.Condition-Lever-R', 
                                                        'ED.Prop-RPM-Np-Eng-L', 
                                                        'ED.Prop-RPM-Np-Eng-R', 
                                                        'ED.GPWS-Alert/Warning', 
                                                        'ED.GPWS-Glideslope-Alert', 
                                                        F.regexp_replace(F.col('ED.AFCS-Glideslope-Active'), '\\*', '').alias("AFCS-Glideslope-Active"),
                                                        'ED.AFCS-LOC-Active', 
                                                        'ED.AFCS-VOR-Active', 
                                                        F.regexp_replace(F.col('ED.GLIDE-PATH-DEV-PILOT-ABOVE'), '\\*', '').alias("GLIDE-PATH-DEV-PILOT-ABOVE"),  
                                                        F.regexp_replace(F.col('ED.GLIDE-PATH-DEV-COPILOT-ABOVE'), '\\*', '').alias("GLIDE-PATH-DEV-COPILOT-ABOVE"),
                                                        'ED.DME-DISTANCE-1', 
                                                        'ED.DME-DISTANCE-2', 
                                                        'ED.VOR-1-SELECT-FREQ', 
                                                        'ED.VOR-2-SELECT-FREQ', 
                                                        'ED.WIND-ANGLE', 
                                                        'ED.WIND-SPEED', 
                                                        'ED.BRAKE-PRESS-LEFT', 
                                                        'ED.BRAKE-PRESS-RIGHT', 
                                                        'ED.Thrust-Rev-Low-Beta-Eng-L', 
                                                        'ED.Thrust-Rev-Low-Beta-Eng-R', 
                                                        'ED.AFCS-Autopilot-Engaged',
                                                        'ED.FileName',
                                                        'ED.LoadedDate',
                                                        'ED.LoadedBy',
                                                        'ED.DataFrameName',
                                                        'ED.year',
                                                        'ED.month' )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
Q400Interim.createOrReplaceTempView('Q400Interim')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
FinalDF = spark.sql(f'''
            
            SELECT 
                    *
            FROM
                    (
                        SELECT 
                                `Time-In-Secs` as `Time-In-Secs-EX`,
                                `Radio-Altitude` AS `Radio-Altitude-EX` ,
                                `Main-WOW` AS `Main-WOW-EX`,
                                `Pitch-Attitude` AS `Pitch-Attitude-EX`,
                                'PitchAttitudeHigh'  as ExceedanceName,
                                DE.Phase,
                                CASE
                                    WHEN
                                            CASE
                                            WHEN
                                                    `Pitch-Attitude` LIKE 'U%'
                                            THEN
                                                    CASE
                                                    WHEN
                                                            INSTR(`Pitch-Attitude`,' ')>0
                                                    THEN
                                                            SUBSTRING(`Pitch-Attitude`, INSTR(`Pitch-Attitude`,' ')+1, LENGTH(`Pitch-Attitude`))
                                                    ELSE
                                                            NULL
                                                    END
                                            END                     > 10.0 
                                    THEN
                                            'High'
                                ELSE
                                        'Low'
                                END TriggerCondition,
                                BC.*
                                
                                            
                                            
                        FROM
                                Q400Interim BC
                        
                        INNER JOIN



                                (
                                    SELECT 
                                            FileName,
                                            Phase,
                                            MAX(`Time-In-Secs`) AS TriggerTime
                                    FROM 
                                        (
                                            SELECT 

                                                    CASE
                                                        WHEN
                                                                `Time-In-Secs` < DJ.MinTime
                                                        THEN
                                                                'TakeOff'
                                                        WHEN
                                                                `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                        THEN
                                                                'Airborne'
                                                        WHEN
                                                                `Time-In-Secs` >DJ.MaxTime
                                                        THEN
                                                                'Approach'
                                                        END Phase,              

                                                    BJ.*

                                            FROM Q400Interim AS BJ

                                            INNER JOIN(

                                    SELECT 

                                    BI.FileName,
                                    BI.`Radio-Altitude`,
                                    MIN(`Time-In-Secs`) as MinTime,
                                    MAX(`Time-In-Secs`) as MaxTime


                                    FROM Q400Interim AS BI
                                    INNER JOIN 
                                        (
                                            SELECT 
                                                    FileName,
                                                    MAX(`Radio-Altitude`) AS MaxRadioHeight
                                            FROM Q400Interim

                                            GROUP BY FileName) T1 
                                    ON BI.FileName=T1.FileName 
                                    AND BI.`Radio-Altitude`=T1.MaxRadioHeight

                                    GROUP BY 
                                        BI.FileName,
                                        BI.`Radio-Altitude` ) DJ 
                        ON BJ.FileName=DJ.FileName ) DD

                WHERE DD.Phase='TakeOff'
                AND    DD.`Main-WOW` = 'Gnd (1)'
                
                GROUP BY
                        DD.FileName,
                        DD.Phase )DE
                ON
                        BC.FileName    = DE.FileName
                AND     BC.`Time-In-Secs` =DE.TriggerTime) FNN
                WHERE
                    FNN.TriggerCondition='High'
                                                            ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
FinalDF.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+-----------------+-----------------+-------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+------------------------+------------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------